In [2]:
import cv2
import numpy as np
import os

#Configuration
main_image_path = 'barbara.jpg'
patch_image_path = 'patch_1.png' # Using patch_1
# Changed output name to reflect edge matching
output_image_path = 'barbara_with_patch_1_edge_match_box.jpg'

#Canny Edge Detector Thresholds
# These thresholds might need tuning for optimal edge detection!
# Lower threshold for linking edges, higher threshold for initial strong edges.
CANNY_THRESHOLD_1 = 100
CANNY_THRESHOLD_2 = 200

print(f"Loading main image: {main_image_path}")
main_img = cv2.imread(main_image_path)
if main_img is None:
    print(f"Error: Could not load main image {main_image_path}")
    exit()

print(f"Loading patch image: {patch_image_path}")
patch_img = cv2.imread(patch_image_path)
if patch_img is None:
    print(f"Error: Could not load patch image {patch_image_path}")
    exit()

print("Converting images to grayscale...")
main_img_gray = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)
patch_img_gray = cv2.cvtColor(patch_img, cv2.COLOR_BGR2GRAY)

# Get Dimensions (use grayscale dimensions)
main_h, main_w = main_img_gray.shape[:2]
patch_h, patch_w = patch_img_gray.shape[:2]

print(f"Main image dimensions (HxW): {main_h} x {main_w}")
print(f"Patch image dimensions (HxW): {patch_h} x {patch_w}")

# Check if patch is larger than the main image
if patch_h > main_h or patch_w > main_w:
    print("Error: Patch image is larger than the main image.")
    exit()

# Apply Canny Edge Detection
print(f"Applying Canny Edge Detection (Thresholds: {CANNY_THRESHOLD_1}, {CANNY_THRESHOLD_2})...")
# You might need to adjust the thresholds 100 and 200 for your specific images
main_edges = cv2.Canny(main_img_gray, CANNY_THRESHOLD_1, CANNY_THRESHOLD_2)
patch_edges = cv2.Canny(patch_img_gray, CANNY_THRESHOLD_1, CANNY_THRESHOLD_2)

# Display edge images (optional, but helpful for debugging/understanding)
cv2.imshow('Main Image Edges', main_edges)
cv2.imshow('Patch Edges', patch_edges)
cv2.waitKey(1) # Give windows time to display


#Search for the Patch Edges in the Main Image Edges (Sliding Window)
# We will count the number of overlapping edge pixels (simple correlation)
max_match_value = -1 # Initialize with a value lower than any possible match count
best_match_coords = (0, 0) # (y, x) of top-left corner

print("Searching for the best edge overlap...")
# Iterate through all possible top-left corners (y, x) for the patch edge map
for y in range(main_h - patch_h + 1):
    for x in range(main_w - patch_w + 1):
        # Extract the current window from the main edge map
        candidate_window_edges = main_edges[y : y + patch_h, x : x + patch_w]

        # Calculate Match Score: Count Overlapping Edge Pixels
        # Method 1: Using bitwise AND and summing
        overlap = cv2.bitwise_and(patch_edges, candidate_window_edges)
        # Canny outputs 0 or 255, so divide by 255 to count pixels
        match_score = np.sum(overlap // 255)

        # Method 2: Simple element-wise multiplication and sum (similar)
        # Since edges are 0 or 255:
        # match_score = np.sum((patch_edges / 255) * (candidate_window_edges / 255))

        # Update if this is a better match (more overlapping edges) 
        if match_score > max_match_value:
            max_match_value = match_score
            best_match_coords = (y, x) # Store NumPy-style coordinates (row, col)

#Get Coordinates for Drawing
best_y, best_x = best_match_coords
top_left = (best_x, best_y)         # OpenCV uses (x, y) for drawing
bottom_right = (best_x + patch_w, best_y + patch_h)

print(f"Best match found for '{patch_image_path}' using Edge Overlap!")
print(f" -> Maximum Overlapping Edge Pixels: {max_match_value}") # Higher overlap is better
print(f" -> Top-left corner (x, y): {top_left}")
print(f" -> Bottom-right corner (x, y): {bottom_right}")

#Draw Bounding Box on Original Image
# Use the original color main_img to draw on
output_img = main_img.copy()
# Using BLUE color for this method's box to distinguish it
box_color = (255, 0, 0) # BGR format for Blue
box_thickness = 2
cv2.rectangle(output_img, top_left, bottom_right, box_color, box_thickness)

#Save and Display Result
print(f"Saving result with bounding box to: {output_image_path}")
cv2.imwrite(output_image_path, output_img)

print("Displaying result. Press any key to close all windows.")
cv2.imshow('Main Image (Original)', main_img)
cv2.imshow('Patch (patch_1.png)', patch_img)
cv2.imshow('Found Patch Location (Edge Match)', output_img) # Updated window title

# Wait for a key press and then close windows
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Done.")

Loading main image: barbara.jpg
Loading patch image: patch_1.png
Converting images to grayscale...
Main image dimensions (HxW): 510 x 510
Patch image dimensions (HxW): 29 x 29
Applying Canny Edge Detection (Thresholds: 100, 200)...
Searching for the best edge overlap...
Best match found for 'patch_1.png' using Edge Overlap!
 -> Maximum Overlapping Edge Pixels: 73
 -> Top-left corner (x, y): (362, 412)
 -> Bottom-right corner (x, y): (391, 441)
Saving result with bounding box to: barbara_with_patch_1_edge_match_box.jpg
Displaying result. Press any key to close all windows.
Done.
